In [1]:
from matplotlib import pyplot as plt
from scipy.special import softmax
from itertools import product
import numpy as np
from utils import *
from model import *
import time
import seaborn as sns
from glob import glob as glob
sns.set_context('talk')

%load_ext autoreload
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
gsname = 'gs0202'

### load data

In [3]:
#
datadf = datadf1 = pd.read_csv('data/%s-datadf-0.csv'%gsname)
# datadf = datadf2 = pd.read_csv('data/%s-datadf-1.csv'%gsname)

In [4]:
gsdf = pd.read_csv('data/%s-summdf.csv'%gsname).astype(float)

In [5]:
## load human data
hdf = pd.read_csv('../human_data.csv')
humanD = {}
for c in ['blocked','interleaved','early','middle','late']:
  humanD[c[0]] = hdf.loc[:,('%s mean')%c].values.T[-40:].mean()
## test acc: difference to human
for c in ['b','i','e','m','l']:
  gsdf['%s-test_diff'%c] = np.abs(gsdf.loc[:,'testacc-%s'%c] - humanD[c])
## mse
gsdf.loc[:,'mse-bi'] = gsdf.loc[:,'mse-i']+gsdf.loc[:,'mse-b']

### plt single condition

In [6]:
def get_cond_df(row):
  ## form query string for above row
  query_str = " and ".join(
    ["%s == %f"%(k,v) for k,v in dict(row.astype(float)[1:10].round(4)).items()]
  )
  print('query',query_str)
  ## df of single parameter condition`
  cond_df = datadf1.round(4).query(query_str)
  if not len(cond_df):
    cond_df = datadf2.round(4).query(query_str)
  paramstr = "__".join(["%s-%s"%(k,v) for k,v in cond_df.iloc[0][1:12].items()])
  print('found',paramstr)
  return cond_df


In [7]:
def plt_cond_df(cond_df,title_extra=''):
  f,axar = plt.subplots(1,1,figsize=(20,8))
  ax=axar
  paramstr = "__".join(["%s-%s"%(k,v) for k,v in cond_df.iloc[0][1:12].items()])
  for c,cdf in cond_df.groupby('cond'):
    try:
      ax.plot(np.arange(200),cdf.acc,label=c)
    except:
      print('err',paramstr,c)
      continue
  ax.set_ylim(0,1)
  plt.legend()
  plt.axhline(0.5,c='k',ls='--')
  plt.title("%s\n%s\nMSE=%s"%(paramstr[:94],paramstr[94:],title_extra))
  plt.savefig('figures/analysis_gs/acc-%s.png'%paramstr)


# top 40 MSE analyses

In [8]:
## select top40 MSE
sgsdf = gsdf.sort_values('mse-bi').iloc[:40]
sgsdf

,Unnamed: 0,concentration,stickiness_wi,stickiness_bt,sparsity,pvar,lrate,lratep,decay_rate,skipt1,...,mse-l,testacc-m,acc2-m,mse-m,b-test_diff,i-test_diff,e-test_diff,m-test_diff,l-test_diff,mse-bi
336,0.0,23.295433,74.287205,78.021423,0.228974,0.0,1.0,1.0,1.0,0.0,...,0.035541,0.573172,0.739577,0.030995,0.059003,0.055956,0.097400,0.117165,0.173822,0.030001
297,0.0,14.571238,75.979485,66.663285,0.228684,0.0,1.0,1.0,1.0,0.0,...,0.047130,0.555484,0.752042,0.038174,0.055629,0.093414,0.086411,0.134853,0.207841,0.030027
1004,0.0,6.768768,99.014263,91.913386,0.099091,0.0,1.0,1.0,1.0,0.0,...,0.050163,0.555778,0.771065,0.039513,0.057918,0.082829,0.061555,0.134559,0.213911,0.030346
110,0.0,9.699376,82.420509,59.125884,0.149487,0.0,1.0,1.0,1.0,0.0,...,0.058174,0.560902,0.772706,0.036833,0.061455,0.102499,0.116342,0.129435,0.232510,0.031659
986,0.0,4.942027,81.639127,40.958096,0.082587,0.0,1.0,1.0,1.0,0.0,...,0.045474,0.543402,0.765084,0.044323,0.062214,0.085163,0.101981,0.146935,0.201603,0.031727
1014,0.0,7.448432,94.244142,84.325245,0.097349,0.0,1.0,1.0,1.0,0.0,...,0.042645,0.596343,0.814616,0.036731,0.061775,0.094005,0.084190,0.093994,0.182484,0.031805
103,0.0,7.773336,83.097686,76.826047,0.137144,0.0,1.0,1.0,1.0,0.0,...,0.046714,0.554718,0.782244,0.044420,0.059696,0.102353,0.059717,0.135619,0.204984,0.031836
665,0.0,9.010291,85.729866,39.061898,0.156227,0.0,1.0,1.0,1.0,0.0,...,0.049056,0.542363,0.774460,0.046518,0.059084,0.114493,0.066990,0.147974,0.210783,0.031908
872,0.0,5.707425,87.546494,77.815790,0.092329,0.0,1.0,1.0,1.0,0.0,...,0.043859,0.521971,0.757358,0.051717,0.051159,0.100348,0.062501,0.168367,0.196459,0.032420
579,0.0,8.049223,86.070492,34.104054,0.144146,0.0,1.0,1.0,1.0,0.0,...,0.049938,0.515271,0.747042,0.052603,0.059762,0.116997,0.071312,0.175066,0.213692,0.033107


In [9]:
# ## plot loops
## TOP MSE PLOTS; modify: save name with rank leading (0-39)
# for idx,row in sgsdf.iterrows():
#   print()
#   cond_df = get_cond_df(row)
#   plt_cond_df(cond_df,str(row.loc['mse-bi']))
#   plt.close('all')